In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
from scipy.optimize import linprog
from tqdm import tqdm

import os
import torch
from scipy.linalg import LinAlgWarning, LinAlgError
from scipy.optimize._optimize import OptimizeWarning
from scipy.optimize._linprog_util import _clean_inputs, _get_Abc
import warnings
import numpy as np

from generate_instances_lp import generate_setcover, Graph, generate_indset, generate_cauctions, generate_capacited_facility_location

In [3]:
rng = np.random.RandomState(1)

In [4]:
from scipy.linalg import qr
from torch_geometric.data import Batch, HeteroData, InMemoryDataset
from collections import namedtuple

_LPProblem = namedtuple('_LPProblem',
                        'c A_ub b_ub A_eq b_eq bounds x0 integrality')
_LPProblem.__new__.__defaults__ = (None,) * 7  # make c the only required arg

In [5]:
def normalize_cons(A, b):
    if A is None or b is None:
        return A, b
    Ab = np.concatenate([A, b[:, None]], axis=1)
    max_logit = np.abs(Ab).max(axis=1)
    max_logit[max_logit == 0] = 1.
    Ab = Ab / max_logit[:, None]
    A = Ab[:, :-1]
    b = Ab[:, -1]
    return A, b

In [387]:
def surrogate_gen(m, n, d):
    A = np.random.randn(m, n)
    A[np.random.rand(m, n) > d] = 0.
    x_feas = np.abs(np.random.randn(n))  # Ensure x_feas is non-negative
    b = A @ x_feas + np.abs(np.random.randn(m)) * 10  # Ensure feasibility

    c = np.abs(np.random.randn(n))
    return A, b, c

In [393]:
A, b, c = surrogate_gen(500, 1000, 0.1)
c = c / (np.abs(c).max() + 1.e-10)  # does not change the result
A, b = normalize_cons(A, b)
bounds = None

In [394]:
m, n = A.shape

In [395]:
# lp = _LPProblem(c, A, b, None, None, None, None, None)
# lp = _clean_inputs(lp)
# A, b, c, *_ = _get_Abc(lp, 0.)

In [396]:
np.linalg.matrix_rank(A)

500

In [397]:
res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
res.x.max()

5.999411316657302

In [398]:
def active_idx(A, x, b, eps=1.e-8):
    vio = A @ res.x - b
    assert vio.max() <= eps
    vio_mask = np.abs(vio) < eps
    return np.where(vio_mask)[0]

# A @ c + b smaller, more likely to be active

In [399]:
actives = active_idx(A, res.x, b)

In [419]:
heur = A @ c / np.linalg.norm(A, ord=None, axis=1) / np.linalg.norm(c) + b
# heur

In [436]:
topk = 40

In [437]:
sortd_Ac_add_b = np.argsort(heur)

In [438]:
preds = np.isin(sortd_Ac_add_b[:topk], actives)

In [439]:
preds.sum() / preds.shape[0]

0.825

In [441]:
preds = np.isin(sortd_Ac_add_b[-topk:], actives)

In [442]:
preds.sum() / preds.shape[0]

0.025

# todo: sensitive analys, for example, add some bias to cons 6, does it change x solution? (6 is not active, but also may affect)

feasible point

# TODO:

split this into l2, l1, log

In [589]:
def find_feas(A, b, c, x0, maxiter = 1000):
    m = np.zeros_like(x0)
    v = np.zeros_like(x0)
    beta1 = 0.9
    beta2 = 0.999
    learning_rate = 0.1
    eps = 1.e-8

    pbar = tqdm(range(maxiter))
    for i in pbar:
        residual = A @ x0 - b
        bias = residual.max()
    
        pbar.set_postfix({'res': residual.max(), 'x': x0.min()})
        if residual.max() <= -1. and x0.min() >= 0.:
            break
        
        # grad = A.T @ np.where(residual > 0, residual, 0.) - np.where(x0 < 0, x0, 0.)
        # grad = A.T @ np.where(residual > 0, 1, 0.) - np.where(x0 < 0, -1, 0.)
        grad = A.T @ (1. / (-residual + bias + 0.1)) - np.where(x0 < 0, x0, 0.)
        m = (1 - beta1) * grad + beta1 * m  # first  moment estimate.
        v = (1 - beta2) * (grad ** 2) + beta2 * v  # second moment estimate.
        mhat = m / (1 - beta1**(i + 1))  # bias correction.
        vhat = v / (1 - beta2**(i + 1))
        x0 = x0 - learning_rate * mhat / (np.sqrt(vhat) + eps)
        x0[x0 < 0] = 0.
    return x0

In [590]:
x = find_feas(A, b, c, np.abs(np.random.randn(c.shape[0])), 1000)

 11%|████████████▉                                                                                                       | 112/1000 [00:00<00:00, 3282.23it/s, res=-1, x=0]


In [591]:
np.argmax(A @ (x - a * c) - b)

5

In [592]:
np.argmax(A @ x - b)

17

In [593]:
A[46]

array([-0.37737332, -0.19583354,  0.85986171, -0.05396768,  0.16993338,
       -0.0949588 ,  0.03225293, -0.16146436, -0.30145799,  0.17564246,
        0.40066374, -0.4507214 , -0.37685126, -0.62093072,  0.01972115,
        0.04681709,  0.26146008,  0.13500878, -0.54775307,  0.31745736,
        0.42852261,  0.27094759, -0.47243575,  0.45955483,  0.06264799,
       -0.31110869, -0.35851797, -0.10525155, -0.01172774,  0.58577606,
        0.10031077,  0.16228011,  0.16750263,  0.55988206,  0.31921448,
       -0.34645189, -0.13155617, -0.42808457,  0.14942064, -0.22317568,
       -0.27262227,  0.43943676, -0.13666641, -0.06876339,  0.10479078,
        0.73070852, -0.40656963, -0.12569153, -0.45336717,  0.0697006 ,
       -0.06502266,  0.25712077,  0.14260872, -0.03153683,  0.44277588,
        0.08419943,  0.02131421, -0.06291146, -0.01877061, -0.17719485,
       -0.46383726, -0.04330475, -0.66471547, -0.00911601, -0.12548365,
       -0.03166618, -0.45075587,  0.13620088,  0.06779389, -0.40

In [ ]:
np.abs()

In [552]:
def find_close_cons(x, A, b, c):
    neg_mask = (-c < 0)
    steps1 = x[neg_mask] / c[neg_mask]
    if len(steps1):
        alpha1 = steps1.min()
    else:
        alpha1 = 1.e8
    
    neg_mask = (A @ c < 0)
    steps2 = (A @ x - b)[neg_mask] / (A @ c - 1.e-10)[neg_mask]
    if len(steps2):
        alpha2 = steps2.min()
    else:
        alpha2 = 1.e8

    a = min(alpha1, alpha2)
    assert (A @ (x - a * c) - b).max() <= 0.
    return a

In [557]:
a = find_close_cons(x, A, b, c)

In [558]:
np.argmax(A @ (x - a * c) - b)

46

In [559]:
np.argmax(A @ x - b)

46

In [560]:
a = find_close_cons(x, A, b, np.eye(x.shape[0])[4])

In [561]:
np.argmax(A @ (x - a * c) - b)

7

In [562]:
np.argmax(A @ x - b)

46

observed problem: if a point is too close to a cons, then its closest cons is pretty much the same as its steepest cons!  

solution 1: find an orthogonal direction (heuristic: find Ai @ eye, argmin(abs))  
solution 2: push it further into the center of the feasible region, use sum log loss func for grad descent